In [1]:
from transformers import GPT2LMHeadModel, GPT2TokenizerFast
from fastai.text.all import *

In [2]:
pretrained_weights = 'gpt2'
tokenizer = GPT2TokenizerFast.from_pretrained(pretrained_weights)
model = GPT2LMHeadModel.from_pretrained(pretrained_weights)

In [3]:
def random_seed(seed_value, use_cuda):
    np.random.seed(seed_value) # cpu vars
    torch.manual_seed(seed_value) # cpu  vars
    random.seed(seed_value) # Python
    if use_cuda: 
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value) # gpu vars
        torch.backends.cudnn.deterministic = True  #needed
        torch.backends.cudnn.benchmark = False
#Remember to use num_workers=0 when creating the DataBunch.

In [4]:
class DropOutput(Callback):
    def after_pred(self): self.learn.pred = self.pred[0]

In [5]:
learn = Learner(dls=None, model=model, loss_func=CrossEntropyLossFlat(), cbs=[DropOutput], metrics=Perplexity()).to_fp16()

### Generated examples from GPT-2 model trained on mixed categories

In [6]:
learn.load("gpt2-finetuned-amazon-reviews-sample-1cycle").cuda()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): Laye

##### Sampling

In [7]:
prompt = "Product looks good but fabric is"

In [8]:
prompt_ids = tokenizer.encode(prompt)
inp = tensor(prompt_ids)[None].cuda()
inp.shape

torch.Size([1, 6])

In [9]:
# set seed to reproduce results
random_seed(seed_value=2021, use_cuda=True)
preds = learn.model.generate(inp, max_length=50, do_sample=True, top_k=0, num_return_sequences=5, temperature=0.7)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [10]:
tokenizer.decode(preds[0].cpu().numpy())

'Product looks good but fabric is not very thick.  I have had this in your car for about 8 years and since it arrived I am very pleased with it.  I have that same car, and it is a very nice car with a good'

In [11]:
for i, pred in enumerate(preds):
  print("{}: {}".format(i, tokenizer.decode(pred, skip_special_tokens=True)))

0: Product looks good but fabric is not very thick.  I have had this in your car for about 8 years and since it arrived I am very pleased with it.  I have that same car, and it is a very nice car with a good
1: Product looks good but fabric is not worth it.I purchased this cd to try to get a taste of its music.  I have read some of its reviews and are going to try and find out which of them is the best cd.  I
2: Product looks good but fabric is a little thin and not strong enough to hold the phone in place. I would recommend this case to anyone who is looking to protect their phone from getting scratched or scratched. I also recommend this case to anyone looking to protect
3: Product looks good but fabric is not strong enough.  I would consider this a real keeper.  I will not use it too often but the material is a little thin.  I will be using it in combination with the small piece of fabric to
4: Product looks good but fabric is not as soft as some of the others.Does what it says it w

##### top-k sampling

In [12]:
prompt = "Product looks good but fabric is"

In [13]:
prompt_ids = tokenizer.encode(prompt)
inp = tensor(prompt_ids)[None].cuda()
inp.shape

torch.Size([1, 6])

In [14]:
# set seed to reproduce results
random_seed(seed_value=2021, use_cuda=True)
preds = learn.model.generate(inp, max_length=50, do_sample=True, top_k=50, num_return_sequences=5, temperature=0.7)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [15]:
tokenizer.decode(preds[0].cpu().numpy())

'Product looks good but fabric is not very thick.  I have had this in a car for about 8 years and it will probably last a long time.  I have a few that are not very durable and those are the ones that I will not'

In [16]:
for i, pred in enumerate(preds):
  print("{}: {}".format(i, tokenizer.decode(pred, skip_special_tokens=True)))

0: Product looks good but fabric is not very thick.  I have had this in a car for about 8 years and it will probably last a long time.  I have a few that are not very durable and those are the ones that I will not
1: Product looks good but fabric is not like it should be.  It is very heavy duty and will not hold up to heavy loads of rain.  I would like to see a smaller fabric to match the quality of the fabric.This is another great
2: Product looks good but fabric is a little thin and not strong enough to hold the phone in place. I would recommend this case to anyone who is looking to protect their phone from getting scratched or scratched. I also recommend this case to anyone looking to protect
3: Product looks good but fabric is not strong enough.  I would consider this a "must have" item for anyone looking for a true long lasting product.I love the flavor of this tea but I will take it apart and see if it tastes better
4: Product looks good but fabric is not as soft as some of the o

##### top-p nucleus sampling

In [17]:
prompt = "Product looks good but fabric is"

In [18]:
prompt_ids = tokenizer.encode(prompt)
inp = tensor(prompt_ids)[None].cuda()
inp.shape

torch.Size([1, 6])

In [19]:
# set seed to reproduce results
random_seed(seed_value=2021, use_cuda=True)
preds = learn.model.generate(inp, max_length=50, do_sample=True, top_k=0, top_p=0.92, num_return_sequences=5, temperature=0.7)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [20]:
tokenizer.decode(preds[0].cpu().numpy())

'Product looks good but fabric is not very thick.  I have had this in a car for about a month and it is still a very good product.I love this cd and have been listening to it for years.  I have no doubt this'

In [21]:
for i, pred in enumerate(preds):
  print("{}: {}".format(i, tokenizer.decode(pred, skip_special_tokens=True)))

0: Product looks good but fabric is not very thick.  I have had this in a car for about a month and it is still a very good product.I love this cd and have been listening to it for years.  I have no doubt this
1: Product looks good but fabric is not like it should be.  It is very heavy duty and will not hold up to heavy loads of rain.  I would like to see a smaller fabric to match the quality of the fabric.This is the best
2: Product looks good but fabric is a little thin and not strong enough to hold the phone in place. I would recommend this case to anyone who is looking for a strong case that does not come with a protective cover. You may want to look elsewhere for
3: Product looks good but fabric is not strong enough.  I would not buy again.I purchased this case for my son and it is an amazing product. I love the look of the case and it fits my son perfectly. He loves it and I
4: Product looks good but fabric is not as soft as some of the others.  I do like the design but I have to

### Generated examples of Toys_and_Games_5

In [6]:
learn.load("gpt2-finetuned-amazon-reviews-sample-1cycle-Toys_and_Games_5").cuda()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): Laye

##### Sampling

In [7]:
prompt = "Amazing toy material is"

In [8]:
prompt_ids = tokenizer.encode(prompt)
inp = tensor(prompt_ids)[None].cuda()
inp.shape

torch.Size([1, 4])

In [9]:
# set seed to reproduce results
random_seed(seed_value=2021, use_cuda=True)
preds = learn.model.generate(inp, max_length=50, do_sample=True, top_k=0, num_return_sequences=5, temperature=0.7)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [10]:
tokenizer.decode(preds[0].cpu().numpy())

'Amazing toy material is soft and sturdy.  I bought this for my 4 year old son as a gift for Christmas.  He loves it.My daughter got this for her 4th birthday and does not seem to mind it. As I said,'

In [11]:
for i, pred in enumerate(preds):
  print("{}: {}".format(i, tokenizer.decode(pred, skip_special_tokens=True)))

0: Amazing toy material is soft and sturdy.  I bought this for my 4 year old son as a gift for Christmas.  He loves it.My daughter got this for her 4th birthday and does not seem to mind it. As I said,
1: Amazing toy material is great and it looks durable.  It is very cute and my child loves it.This is a super cool set of dice for my 4 yo. This is a great set of dice, great for the grandsons, kids
2: Amazing toy material is amazing.  I love it and I purchased the new X-Wing X-Wing X-wing X-Wing X-Wing X-Wing X-Wing X-Wing X-Wing X-Wing X-Wing X-
3: Amazing toy material is durable, very cute, very durable, and a great buy.Very good condition.This is one of my favorite toys.  It's very easy to put together and the pieces are very sturdy, very durable.  It has
4: Amazing toy material is very durable. My son loves it. He loves it. He wants a ball. His toys are so colorful and cute. I'm very impressed with this toy! I would suggest this toy.Granddaughter loves this kit!!!I


##### top-k sampling

In [12]:
prompt = "Amazing toy material is"

In [13]:
prompt_ids = tokenizer.encode(prompt)
inp = tensor(prompt_ids)[None].cuda()
inp.shape

torch.Size([1, 4])

In [14]:
# set seed to reproduce results
random_seed(seed_value=2021, use_cuda=True)
preds = learn.model.generate(inp, max_length=50, do_sample=True, top_k=50, num_return_sequences=5, temperature=0.7)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [15]:
tokenizer.decode(preds[0].cpu().numpy())

"Amazing toy material is soft and sturdy.  I bought this for my 4 year old son as a gift for Christmas.  He loves it.My daughter got this for her 4th birthday and loves it.  It's sturdy.Very well made"

In [16]:
for i, pred in enumerate(preds):
  print("{}: {}".format(i, tokenizer.decode(pred, skip_special_tokens=True)))

0: Amazing toy material is soft and sturdy.  I bought this for my 4 year old son as a gift for Christmas.  He loves it.My daughter got this for her 4th birthday and loves it.  It's sturdy.Very well made
1: Amazing toy material is great and it looks durable.  It is very cute and my child loves it.This is a super cool set of dice for my 4 yo. This is a great set of dice, great for the price, and very functional
2: Amazing toy material is amazing.  I love it and I purchased the new X-Wing X-Wing X-wing X-Wing X-Wing X-Wing X-Wing X-Wing X-Wing X-Wing X-Wing X-
3: Amazing toy material is durable, very cute, very durable, and a great buy.Very good condition.This is one of my favorite toys.  It's very easy to put together and the pieces are very sturdy, very durable.  It has
4: Amazing toy material is very durable. My son loves it. He loves it. He wants a ball. His toys are so colorful and cute. I'm very impressed with this toy! I would suggest this toy.Granddaughter loves this kit!!!I


##### top-p nucleus sampling

In [17]:
prompt = "Amazing toy material is"

In [18]:
prompt_ids = tokenizer.encode(prompt)
inp = tensor(prompt_ids)[None].cuda()
inp.shape

torch.Size([1, 4])

In [19]:
# set seed to reproduce results
random_seed(seed_value=2021, use_cuda=True)
preds = learn.model.generate(inp, max_length=50, do_sample=True, top_k=0, top_p=0.92, num_return_sequences=5, temperature=0.7)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [20]:
tokenizer.decode(preds[0].cpu().numpy())

'Amazing toy material is soft and sturdy.  I bought this for my 4 year old son and he loves it.  I am sure he will enjoy it as well.  We have a lot of fun with it and it was a huge hit!'

In [21]:
for i, pred in enumerate(preds):
  print("{}: {}".format(i, tokenizer.decode(pred, skip_special_tokens=True)))

0: Amazing toy material is soft and sturdy.  I bought this for my 4 year old son and he loves it.  I am sure he will enjoy it as well.  We have a lot of fun with it and it was a huge hit!
1: Amazing toy material is great and it looks great.  It is very sturdy and has been used for over a year now.  I really like this toy.  It is well made and the quality is good.  I think it is very well
2: Amazing toy material is amazing.  I love it and I'm so excited to buy more!I got this for my son who is a huge fan of the Marvel series and has been loving it for years.  He loves it, has played with
3: Amazing toy material is durable, very cute, very durable, and a great buy.Very good quality.This is one of my favorite toys.  It's very easy to put together and the pieces are very sturdy.  The first time I got
4: Amazing toy material is very durable. My son loves it. He loves it. He wants to play with it again. I recommend it.Love this!  Great price!These are very cute.  My daughter loves them.  She

### Generated examples of Tools_and_Home_Improvement_5

In [7]:
learn.load("gpt2-finetuned-amazon-reviews-sample-1cycle-Tools_and_Home_Improvement_5").cuda()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): Laye

##### Sampling

In [8]:
prompt = "Amazing tool"

In [9]:
prompt_ids = tokenizer.encode(prompt)
inp = tensor(prompt_ids)[None].cuda()
inp.shape

torch.Size([1, 2])

In [10]:
# set seed to reproduce results
random_seed(seed_value=2021, use_cuda=True)
preds = learn.model.generate(inp, max_length=50, do_sample=True, top_k=0, num_return_sequences=5, temperature=0.7)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [11]:
tokenizer.decode(preds[0].cpu().numpy())

"Amazing tool, very handy!  I have had this in your car for years and it is a very nice tool.This is a great little light bulb and that's why I gave it 5 stars.  It's bright and well packed. "

In [12]:
for i, pred in enumerate(preds):
  print("{}: {}".format(i, tokenizer.decode(pred, skip_special_tokens=True)))

0: Amazing tool, very handy!  I have had this in your car for years and it is a very nice tool.This is a great little light bulb and that's why I gave it 5 stars.  It's bright and well packed. 
1: Amazing tool for the money.I purchased two of these to replace yin and yang pliers. I have read the review that says they are going to be big enough to cut through concrete, concrete, granite and anything else that would require a
2: Amazing tool for a small toolbox.I use this often in my garage.  I use it every day.  I have to remove the plastic cover from the wall and pop it out.  The glue on the cover has a lot of dust
3: Amazing tool, but I don't think the blade is a real keeper. It's just a bit too sharp to hold the metal much better, and if you're cutting through metal, the chuck is a bit too sensitive for that.

The
4: Amazing tool.  I use it every day just to see what the weather is doing and stay on track when I need to.  I'm very impressed with this tool! I would recommend it!This i

##### top-k sampling

In [13]:
prompt = "Amazing tool"

In [14]:
prompt_ids = tokenizer.encode(prompt)
inp = tensor(prompt_ids)[None].cuda()
inp.shape

torch.Size([1, 2])

In [15]:
# set seed to reproduce results
random_seed(seed_value=2021, use_cuda=True)
preds = learn.model.generate(inp, max_length=50, do_sample=True, top_k=50, num_return_sequences=5, temperature=0.7)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [16]:
tokenizer.decode(preds[0].cpu().numpy())

'Amazing tool, very handy!  I have had this in every car ever since I installed it.  This seems to be very well constructed, well made and works as advertised.  The handle is very solid and sturdy.  The base is a'

In [17]:
for i, pred in enumerate(preds):
  print("{}: {}".format(i, tokenizer.decode(pred, skip_special_tokens=True)))

0: Amazing tool, very handy!  I have had this in every car ever since I installed it.  This seems to be very well constructed, well made and works as advertised.  The handle is very solid and sturdy.  The base is a
1: Amazing tool for the money.I purchased two of these to replace my other faucets.  The first one was an "off-white" one and it was not bright enough to light my bathroom.  The second one was a black one
2: Amazing tool for a small toolbox.I use this often in my garage.  I use it every day.  I have to remove the plastic cover from the wall and it is hard to remove, but I don't have to do it again
3: Amazing tool, but I don't think the blade is a real problem. It's just a bit larger than the tool the first time I used it. I'm sure that it will take some time to get used to first use, but it's
4: Amazing tool.  I use it every day just to see what the weather is doing and I have no complaints.  I really like the price and it works well.  I would recommend it!This is a very goo

##### top-p nucleus sampling

In [18]:
prompt = "Amazing tool"

In [19]:
prompt_ids = tokenizer.encode(prompt)
inp = tensor(prompt_ids)[None].cuda()
inp.shape

torch.Size([1, 2])

In [20]:
# set seed to reproduce results
random_seed(seed_value=2021, use_cuda=True)
preds = learn.model.generate(inp, max_length=50, do_sample=True, top_k=0, top_p=0.92, num_return_sequences=5, temperature=0.7)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [21]:
tokenizer.decode(preds[0].cpu().numpy())

'Amazing tool, very handy!  I have had this in every car ever since I installed it.  This is a perfect tool for the job, perfect for that job.  The leatherman is a little stiff, but this is the best I'

In [22]:
for i, pred in enumerate(preds):
  print("{}: {}".format(i, tokenizer.decode(pred, skip_special_tokens=True)))

0: Amazing tool, very handy!  I have had this in every car ever since I installed it.  This is a perfect tool for the job, perfect for that job.  The leatherman is a little stiff, but this is the best I
1: Amazing tool for the money.I purchased two of these to replace my other faucets.  The first one was an "off-white" one and it was not bright enough to light my bathroom.  The second one was a black one
2: Amazing tool for a small toolbox.I use this as a review.  I have been using it for about a year now.  It works great for the price and has been used for a very long time.  It's a great way
3: Amazing tool, but I don't think the blade is a real keeper. It's just a bit too sharp for my needs. I'll give it 5 stars if it's worth the price.great product, great price for a tool.I use
4: Amazing tool.  I use it every day.  I use it to remove my old power tool and replace it with this one.  I've been using it for years.  I'm glad I did.  I think I'll buy another


### Generated examples of Pet_Supplies_5

In [6]:
learn.load("gpt2-finetuned-amazon-reviews-sample-1cycle-Pet_Supplies_5").cuda()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): Laye

#####  Sampling

In [7]:
prompt = "Taste is good but packaging was poor but my"

In [8]:
prompt_ids = tokenizer.encode(prompt)
inp = tensor(prompt_ids)[None].cuda()
inp.shape

torch.Size([1, 10])

In [9]:
# set seed to reproduce results
random_seed(seed_value=2021, use_cuda=True)
preds = learn.model.generate(inp, max_length=50, do_sample=True, top_k=0, num_return_sequences=5, temperature=0.7)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [10]:
tokenizer.decode(preds[0].cpu().numpy())

'Taste is good but packaging was poor but my dog loves it!  I bought this for her in exchange for a review.  I will buy again.This is a great product, very easy to assemble and is easy to use. My dog'

In [11]:
for i, pred in enumerate(preds):
  print("{}: {}".format(i, tokenizer.decode(pred, skip_special_tokens=True)))

0: Taste is good but packaging was poor but my dog loves it!  I bought this for her in exchange for a review.  I will buy again.This is a great product, very easy to assemble and is easy to use. My dog
1: Taste is good but packaging was poor but my dog wasn't spoiled. I bought the medium and it fits my dog perfectly and fits her nice. She's an aggressive chewer and I'm sure it feels good on the dog. I'll continue
2: Taste is good but packaging was poor but my dogs have not used it.I use this often in my kennel.  I usually vacuum it out in the yard so it makes a mess if you have a big dog.  But this
3: Taste is good but packaging was poor but my puppy is very happy with it.  I wish it had a pine flavor.My cat can get the brush out of the bag and have no problem with it. It is a little hard to get
4: Taste is good but packaging was poor but my dogs love it.My cats love the taste.  They like the taste and stay away from the other cat food.  I'm also noticed that they prefer the other food

##### top-k sampling

In [12]:
prompt = "Taste is good but packaging was poor but my"

In [13]:
prompt_ids = tokenizer.encode(prompt)
inp = tensor(prompt_ids)[None].cuda()
inp.shape

torch.Size([1, 10])

In [14]:
# set seed to reproduce results
random_seed(seed_value=2021, use_cuda=True)
preds = learn.model.generate(inp, max_length=50, do_sample=True, top_k=50, num_return_sequences=5, temperature=0.7)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [15]:
tokenizer.decode(preds[0].cpu().numpy())

'Taste is good but packaging was poor but my dog loves it!  I bought this for her in exchange for a review.  I will buy again.This is a great product, very easy to assemble and is easy to use. My dog'

In [16]:
for i, pred in enumerate(preds):
  print("{}: {}".format(i, tokenizer.decode(pred, skip_special_tokens=True)))

0: Taste is good but packaging was poor but my dog loves it!  I bought this for her in exchange for a review.  I will buy again.This is a great product, very easy to assemble and is easy to use. My dog
1: Taste is good but packaging was poor but my dog wasn't thrilled. I bought the medium and it fits my dog perfectly and fits her nice. She's an aggressive chewer and I'm sure it feels good on the dog. I'll continue
2: Taste is good but packaging was poor but my dogs have not used it.I use this as a treat for my two kittens. They love it. They play with it every day. It is easy for them to use. It's not too
3: Taste is good but packaging was poor but my puppy is very happy with it.  I wish it had a little more weight since she can get the tummy to rest on the top of the bag.  It will take her some time to
4: Taste is good but packaging was poor but my dogs love it.My cats love the taste.  They like the taste and I have a picky cat that really gets excited when I shake it out.  I would su

##### top-p nucleus sampling

In [17]:
prompt = "Taste is good but packaging was poor but my"

In [18]:
prompt_ids = tokenizer.encode(prompt)
inp = tensor(prompt_ids)[None].cuda()
inp.shape

torch.Size([1, 10])

In [19]:
# set seed to reproduce results
random_seed(seed_value=2021, use_cuda=True)
preds = learn.model.generate(inp, max_length=50, do_sample=True, top_k=0, top_p=0.92, num_return_sequences=5, temperature=0.7)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [20]:
tokenizer.decode(preds[0].cpu().numpy())

'Taste is good but packaging was poor but my dog loves it!  I bought this for her in exchange for a review.  I will buy again.This is a great product, very easy to assemble and is easy to use. My dog'

In [21]:
for i, pred in enumerate(preds):
  print("{}: {}".format(i, tokenizer.decode(pred, skip_special_tokens=True)))

0: Taste is good but packaging was poor but my dog loves it!  I bought this for her in exchange for a review.  I will buy again.This is a great product, very easy to assemble and is easy to use. My dog
1: Taste is good but packaging was poor but my dog wasn't thrilled. I bought the medium and it fits my dog perfectly and fits her perfectly. She's an aggressive chewer and I'm sure it's not the same quality as the standard size
2: Taste is good but packaging was poor but my dogs have not used it.I use this as a treat for my two kittens. They love it. They play with it every day. It is easy to put together and the other cats don't
3: Taste is good but packaging was poor but my puppy is very happy with it.  I wish it had a little more weight to it.This is an awesome product. I love the flavor. It's easy to clean, easy to put on
4: Taste is good but packaging was poor but my dogs love it.My cats love it.  They do not like it but it's so much easier to give to them and keep them from chewing